# Workbook to maintain work ticks

This notebook uses C# interactive shell and SQL API of CosmosDB. It is used to maintain the records of different project execution timer.


## Declaring constants for the notebook


In [7]:
public const string databaseId = "manual-log";
public const string containerId = "aws-entries";

## Initializing the Cosmos client and container




In [3]:
using System;
using Microsoft.Azure.Cosmos;
using System.Collections;
using System.Net;
using Newtonsoft.Json;

// Initialize a new instance of CosmosClient using the built-in account endpoint and key parameters
CosmosClient cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);

Database dataHost = cosmosClient.GetDatabase(databaseId);
Container itemHost = dataHost.GetContainer(containerId);

## Defining the POCO for item



In [4]:
public class LogItem {

    [JsonProperty(PropertyName="id")]
    public string Id {get;set;}

    [JsonProperty(PropertyName="record-name")]
    public string RecordName {get;set;}

    [JsonProperty(PropertyName="tag")]
    public string Tag {get;set;}

    [JsonProperty(PropertyName="start-timestamp")]
    public DateTime? StartedAt {get;set;}

    [JsonProperty(PropertyName="end-timestamp")]
    public DateTime? EndedAt {get;set;}

    [JsonProperty(PropertyName="comment")]
    public string Comment {get;set;}

    [JsonProperty(PropertyName="resource-url")]
    public string ResourceUrl {get;set;}

    [JsonProperty(PropertyName="service")]
    public string ServiceName {get;set;}

    public override string ToString() {
        return $"Log entry is with id- {Id}, name \"{RecordName}\", and started at {StartedAt.ToString()}";
    }
}

## Put data into the structure

This code block should be changed every time you want a new record to be inserted.



In [10]:
LogItem newEntry = new LogItem(){
    Id = "5",
    RecordName = "EC2 - Getting started tutorial",
    Tag = "Aws-Certification",
    StartedAt = DateTime.UtcNow,
    EndedAt = null,
    Comment = "",
    ResourceUrl = "https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/EC2_GetStarted.html",
    ServiceName = "EC2"
};

## Add the item to container

**CAUTION** skip this code block if you want to only update.


In [2]:
ItemResponse<LogItem> response = await itemHost.CreateItemAsync<LogItem>(newEntry);

Display.AsMarkdown($"Charges of query - **{response.RequestCharge}**");
Display.AsMarkdown($"` {response.Resource.ToString()} `");

Charges of query - **8**

` Log entry is with id- 5, name "EC2 - Getting started tutorial", and started at 1/11/2021 5:37:46 AM `

## Querying back the item



In [3]:
ItemResponse<LogItem> queryResponse = await itemHost.ReadItemAsync<LogItem>(

    partitionKey: new PartitionKey("EC2"),
    id: "5"
);

Display.AsMarkdown($"Retreived document: `{queryResponse.Resource}`");
Display.AsMarkdown($"#### Diagnostics for ReadItemAsync:");
Display.AsMarkdown($"` {queryResponse.Diagnostics.ToString()} `");



Retreived document: `Log entry is with id- 5, name "EC2 - Getting started tutorial", and started at 1/11/2021 5:37:46 AM`

#### Diagnostics for ReadItemAsync:

` {"DiagnosticVersion":"2","Summary":{"StartUtc":"2021-01-11T06:20:06.0845733Z","TotalElapsedTimeInMs":150.2675,"UserAgent":"cosmos-netstandard-sdk/3.12.0|3.11.4|10|X64|Linux 4.19.90-microsoft-standa|.NET Core 3.1.3|","TotalRequestCount":1,"FailedRequestCount":0},"Context":[{"Id":"Microsoft.Azure.Cosmos.Handlers.DiagnosticsHandler","HandlerElapsedTimeInMs":146.2215},{"Id":"SystemInfo","CpuHistory":"(2021-01-11T06:19:12.2952831Z 0.000), (2021-01-11T06:19:12.2960783Z 100.000), (2021-01-11T06:19:22.3021369Z 1.587), (2021-01-11T06:19:42.3061923Z 0.000), (2021-01-11T06:19:52.2991621Z 0.000), (2021-01-11T06:20:02.2974400Z 0.000)"},{"Id":"Microsoft.Azure.Cosmos.Handlers.RetryHandler","HandlerElapsedTimeInMs":146.2179},{"Id":"Microsoft.Azure.Cosmos.Handlers.RouterHandler","HandlerElapsedTimeInMs":146.21},{"Id":"Microsoft.Azure.Cosmos.Handlers.TransportHandler","HandlerElapsedTimeInMs":0.1412},{"Id":"AggregatedClientSideRequestStatistics","ContactedReplicas":[],"RegionsContacted":["https://project-timers-eastus.documents.azure.com:443/"],"FailedReplicas":[]},{"Id":"Microsoft.Azure.Documents.ServerStoreModel","ElapsedTimeInMs":146.1352},{"Id":"AddressResolutionStatistics","StartTimeUtc":"2021-01-11T06:20:06.1796911Z","EndTimeUtc":"2021-01-11T06:20:06.1882037Z","ElapsedTimeInMs":8.5126,"TargetEndpoint":"https://project-timers-eastus.documents.azure.com//addresses/?$resolveFor=dbs%2fIKltAA%3d%3d%2fcolls%2fIKltAPKrgKU%3d%2fdocs&$filter=protocol eq rntbd&$partitionKeyRangeIds=0"},{"Id":"StoreResponseStatistics","StartTimeUtc":"2021-01-11T06:20:06.0848898Z","ResponseTimeUtc":"2021-01-11T06:20:06.2309397Z","ElapsedTimeInMs":146.0499,"ResourceType":"Document","OperationType":"Read","LocationEndpoint":"https://project-timers-eastus.documents.azure.com:443/","ActivityId":"ef4d8748-8b07-4ef4-bcf6-86f62133fbc3","StoreResult":"StorePhysicalAddress: rntbd://cdb-ms-prod-eastus1-fd49.documents.azure.com:14145/apps/8acd1527-a022-4c2a-b2d9-2eea6c3b221d/services/f085975e-f496-48a5-ae43-5ce81ca5fab8/partitions/05064715-9b8e-4a86-8aab-236ca97ea635/replicas/132547803045701812s/, LSN: 8, GlobalCommittedLsn: 7, PartitionKeyRangeId: 0, IsValid: True, StatusCode: 200, SubStatusCode: 0, RequestCharge: 1, ItemLSN: 8, SessionToken: -1#8, UsingLocalLSN: True, TransportException: null"}]} `

## Updating the EndDateTimeStamp for the record


In [4]:
LogItem endItem = (LogItem) queryResponse;

endItem.EndedAt = DateTime.UtcNow;

ItemResponse<LogItem> upsertResponse = await itemHost.UpsertItemAsync(partitionKey: new PartitionKey("EC2"), item: endItem);
Display.AsMarkdown($"Charges of query: **{upsertResponse.RequestCharge}**");

Charges of query: **10.86**